In [16]:
# some basic estimations
import numpy as np
import astropy.units as u
from astropy import constants as const
from scipy.integrate import simps
from scipy.special import hyp1f1

m_e = (const.m_e.cgs).value
c= (const.c.cgs).value
e = (const.e.esu).value
sigma_sb = (const.sigma_sb.cgs).value
pi = np.pi
Lam_max = (1e18*(u.cm)).value

# cut off energy between stochastic and shear acceleration
def gamma_eq(lgB,Reg,beta,lgN,lgrho):
    B0=(((10**lgB)*(u.uG)).to(u.G)).value
    rho = 10**lgrho # jet density(g/cm^3)
    xi=1
    q=5/3
    Delta_r = ((Reg*(u.pc)).cgs).value
    Gamma_j = (1/(4*np.square(beta)))*np.square(np.log((1+beta)/(1-beta))) #averaged
    sigma_e = (8*pi/3)*np.square(e**2/(m_e*c**2))
    beta_Alf = (B0/np.sqrt(4*pi*rho))/c # Velocity of Alfven wave 
    Gamma_Alf = 1/np.sqrt(1-beta_Alf**2)
    Grad = beta/Delta_r

    gamma_c = (((15*(q+2))/(2*(6-q)))*(xi**2*Gamma_Alf**4*beta_Alf**2/(Lam_max**(2*q-2)*Gamma_j**4*Grad**2)))**(1/(4-2*q))*(e*B0/(m_e*np.square(c)))
    E_eq = ((gamma_c*m_e*c**2)*(u.erg)).to(u.eV)

    return E_eq.value

# Single shearing electron spectrum, above 1 TeV to cutoff energy
def SEPL(lgB,Reg,beta,lgN):
    B0=(((10**lgB)*(u.uG)).to(u.G)).value
    Reg_sh = ((Reg*(u.pc)).cgs).value
    beta = beta
    N_tot = 10**lgN # Normalization index, 10**Norm_A
    Lam_max = (1e18*(u.cm)).value
    z=0
    xi=1
    q=5/3

    Gamma_j = (1/(4*np.square(beta)))*np.square(np.log((1+beta)/(1-beta))) # averaged bulk lorentz factor
    Grad = beta/Reg_sh
    D_sh = (2/15)*Gamma_j*np.square(c)*np.square(Grad)
    sigma_e = (8*np.pi/3)*np.square(e**2/(m_e*c**2)) # cross section of electrons
    T_cmb = 2.72 # CMB temperature
    U_B = np.square(B0)/8*np.pi

    U_cmb = ((4*sigma_sb)/c)*T_cmb**4*(1+z)**4 # only CMB is considered
    U_rad = U_cmb

    A1 = D_sh*xi**(-1)*(Lam_max/c)**(q-1)*(m_e*c/(e*B0))**(2-q)
    A2 = (sigma_e*B0**2*(1+U_rad/U_B))/(6*np.pi*m_e*c)
    gamma_max = (((6-q)/2)*(A1/A2))**(1/(q-1))
    gamma_cut = 10*gamma_max # n(10γ_max)=0
    E_cut=(((gamma_cut*m_e*c**2)*(u.erg)).to(u.eV)).value
    lgcut = np.log10(E_cut)

    if(lgcut<=12):
        return np.zeros(1)-np.inf
    
    z_cut=((6-q)/(1-q))*(gamma_cut/gamma_max)**(q-1)
    w = 40/np.square(np.log((1+beta)/(1-beta)))
    s1 = (q-1)/2+np.sqrt((5-q)**2/4+w)
    s2 = (q-1)/2-np.sqrt((5-q)**2/4+w)
    a1 = (2+s1)/(q-1)
    a2 = (2+s2)/(q-1)
    b1 = (2*s1)/(q-1)
    b2 = (2*s2)/(q-1)
    C2 = 1
    C1 = -C2*gamma_cut**(s2-s1)*(hyp1f1(a2,b2,z_cut)/hyp1f1(a1,b1,z_cut))

    E_all = (np.logspace(12,lgcut,1000)*(u.eV)).to(u.erg) # selected energy range
    gamma_all = (E_all.value)/(m_e*np.square(c)) #10**np.arange(min,max,0.1)
    mask = np.where(gamma_all<=gamma_cut) # cutoff energy
    gamma_all = gamma_all[mask]
    z=((6-q)/(1-q))*(gamma_all/gamma_max)**(q-1)
    n_all = C1*gamma_all**s1*hyp1f1(a1,b1,z)+C2*gamma_all**s2*hyp1f1(a2,b2,z)
    mask1 = np.where(n_all>0)
    n_all=n_all[mask1]
    gamma_all = gamma_all[mask1]
    #ene_eV = ((gamma_all*m_e*c**2)*(u.erg)).to(u.eV) # energy in eV
    #n_eV = n_all/((m_e*c**2)*(u.erg)).to(u.eV) # number density(1/eV)
    length = len(n_all)
    if (length<=5):
        return np.zeros(1)-np.inf
    
    N_A = simps(n_all, gamma_all)
    n_all=N_tot*n_all/N_A

    return gamma_all,n_all

# Jet luminosity, inputs and outputs should both in cgs
def Jet_L(gamma_all,n_all,lgB):
    B0=(((10**lgB)*(u.uG)).to(u.G)).value
    sigma_e = (8*np.pi/3)*np.square(e**2/(m_e*c**2))
    T_cmb = 2.72 # CMB temperature
    U_B = np.square(B0)/8*np.pi
    z=0
    U_cmb = ((4*sigma_sb)/c)*T_cmb**4*(1+z)**4 # only CMB is considered
    U_rad = U_cmb
    f=U_rad/U_B
    A2 = (sigma_e*np.square(B0)*(1+f))/(6*pi*m_e*c)
    t_cool = (A2*gamma_all)**(-1)
    x = gamma_all
    y = m_e*c**2*(gamma_all*n_all/t_cool)
    L_jet = simps(y,x)
    return L_jet

# Combined spectrum of stochastic and shear acceleration
def SSEPL(lgB,Reg,beta,lgN,lgrho):
    B0=(((10**lgB)*(u.uG)).to(u.G)).value
    Reg_sh = ((Reg*(u.pc)).cgs).value
    beta = beta
    N_tot = 10**lgN # Normalization index, 10**Norm_A
    rho = 10**lgrho
    Lam_max = (1e18*(u.cm)).value
    z=0
    xi=1
    q=5/3
    Delta_r = Reg_sh
    Gamma_j = (1/(4*np.square(beta)))*np.square(np.log((1+beta)/(1-beta))) # averaged bulk lorentz factor
    Grad=beta/Delta_r
    
    # calculate the maximum energy for stochastic acc
    beta_Alf = (B0/np.sqrt(4*pi*rho))/c # Alfven velocity
    Gamma_Alf = 1/np.sqrt(1-beta_Alf**2)
    gamma_st = ((15*(q+2)/(2*(6-q)))*(Gamma_Alf**2*(np.square(xi*Gamma_Alf*beta_Alf))/(Lam_max**(2*q-2)*np.square(Gamma_j**2*Grad))))**(1/(4-2*q))*(e*B0/(m_e*np.square(c)))
    E_st = (((gamma_st*m_e*c**2)*(u.erg)).to(u.eV)).value
    lgE_st = np.log10(E_st)
    
    # This part is for the shearing spectrum
    D_sh = (2/15)*Gamma_j*np.square(Grad*c)
    sigma_e = (8*np.pi/3)*np.square(e**2/(m_e*c**2)) # cross section of electrons
    T_cmb = 2.72 # CMB temperature
    U_B = np.square(B0)/8*np.pi
    U_cmb = ((4*sigma_sb)/c)*T_cmb**4*(1+z)**4 # only CMB is considered
    U_rad = U_cmb
    A1 = D_sh*xi**(-1)*(Lam_max/c)**(q-1)*(m_e*c/(e*B0))**(2-q)
    A2 = (sigma_e*B0**2*(1+U_rad/U_B))/(6*np.pi*m_e*c)
    gamma_max = (((6-q)/2)*(A1/A2))**(1/(q-1))
    gamma_cut = 10*gamma_max # n(10γ_max)=0
    E_cut=(((gamma_cut*(m_e*c**2))*(u.erg)).to(u.eV)).value
    lgE_cut = np.log10(E_cut)
    z_cut=((6-q)/(1-q))*(gamma_cut/gamma_max)**(q-1)
    w = 40/np.square(np.log((1+beta)/(1-beta)))
    s1 = (q-1)/2+np.sqrt((5-q)**2/4+w)
    s2 = (q-1)/2-np.sqrt((5-q)**2/4+w)
    a1 = (2+s1)/(q-1)
    a2 = (2+s2)/(q-1)
    b1 = (2*s1)/(q-1)
    b2 = (2*s2)/(q-1)
    C2 = 1
    C1 = -C2*gamma_cut**(s2-s1)*(hyp1f1(a2,b2,z_cut)/hyp1f1(a1,b1,z_cut))

    # electron spec for shearing acc, the spectrum should begin at lower energies to ensure the number of particles
    if(lgE_st<=9) or (lgE_st>=lgE_cut):
        return np.zeros(1)-np.inf
    
    E_sh = (np.logspace(lgE_st,lgE_cut,1000)*(u.eV)).to(u.erg) # selected energy range(TeV)
    gamma_sh = (E_sh.value)/(m_e*np.square(c)) 
    mask = np.where(gamma_sh<=gamma_cut) # cutoff energy
    gamma_sh = gamma_sh[mask]
    z=((6-q)/(1-q))*(gamma_sh/gamma_max)**(q-1)
    n = C1*gamma_sh**s1*hyp1f1(a1,b1,z)+C2*gamma_sh**s2*hyp1f1(a2,b2,z)
    mask1 = np.where(n>0)
    n=n[mask1]
    gamma_sh = gamma_sh[mask1]

    # electron spec for stochastic acc
    E_low = (np.logspace(9,lgE_st,1000)*(u.eV)).to(u.erg)
    gamma_low = (E_low.value)/(m_e*np.square(c))
    N0 = n[0]/(gamma_low[-1])**(1-q)
    n_low = N0*gamma_low**(1-q)

    # Connect the spectrum and do the normalization

    gamma_all = np.append(gamma_low[:-1],gamma_sh)
    n_all = np.append(n_low[:-1],n)

    #ene_eV = ((gamma_all*m_e*c**2)*(u.erg)).to(u.eV) # energy in eV
    #n_eV = n_all/((m_e*c**2)*(u.erg)).to(u.eV) # number density(1/eV)

    N_A = simps(n_all, gamma_all)
    n_all=N_tot*n_all/N_A
    length = len(n_all)
    if (length<=5):
        return np.zeros(1)-np.inf

    #EC = TableModel(ene_all,n_all)
    #IC = InverseCompton(EC,seed_photon_fields=['CMB'],Eemax = 1e18*(u.eV))
    #spectrum_energy_ic = ene_obs*(u.TeV) # observed energies
    #sed_IC = IC.sed(spectrum_energy_ic,distance=6.6*u.kpc) # erg/s
    ##sed_IC = sed_IC.value 
    #sed_IC=N_tot*(sed_IC.value) # number of particles:10**lgN
   
    return gamma_all,n_all

def Kine_L(gamma_all,n_all,beta):
    N_tot = simps(n_all,gamma_all)
    L = ((50*(u.pc)).cgs).value
    Gamma = 1/np.sqrt(1-beta**2)
    Kine_L = beta*c*Gamma**2*N_tot*m_e*c**2/L
    return Kine_L

def Density_est(gamma_all,n_all,Reg):
    N_tot = simps(n_all,gamma_all)
    M = N_tot*((const.m_p).cgs).value
    V = ((50*(u.pc)).cgs).value*(((Reg*(u.pc)).cgs).value)**2
    lgrho_est = np.log10(M/V)
    return lgrho_est


In [ ]:
lgB=0.998
Reg=2.35
beta=0.92
lgN=49.12
lgrho=-28
E_eq=gamma_eq(lgB,Reg,beta,lgN,lgrho)
print("Energy(eV)%e"%E_eq)
gamma_all,n_all= SSEPL(lgB,Reg,beta,lgN,lgrho)
L_jet = Jet_L(gamma_all,n_all,lgB)
print('Jet luminosity(erg/s):%s'%L_jet)
N_electrons=simps(n_all,gamma_all)
print('Total number of electrons:%s'% N_electrons)
Kine_L =Kine_L(gamma_all,n_all,beta)
print('Kinetic luminosity(erg/s):%s'% Kine_L)
rho_est = Density_est(gamma_all,n_all,Reg)
print('Average log density:%s'%rho_est)

2.796153e+13
Jet luminosity(erg/s):2.1229801572526108e+40
Total number of electrons:1.3182567385563993e+49
Kinetic luminosity(erg/s):1.25610853481644e+34
Average log density:-32.56575957959733
